## **Decorators & higher order functions**
---
When I first got confronted with decorators in Python I was quite confused with the concept. Coming from Java I had of course seen a decorator `@override` which is Java to say you have overridden some method specified in an interface, in Java these are called annotations. Of course, you can implement the decorator programming pattern in Java, which Wikipedia tells me not to confuse with a Python decorator, I find that too strict. The decorator pattern adds behaviour to a single object, Python decorators adds behaviour to foremostly functions/methods, but can also add /modify behaviour of an entire class, as we will see in this notebook. You could easily program a structure that given some flag creates a decorated or regular object, therefore doing exactly what the decorator programming pattern does. 

So, from a practical view both the decorator pattern and Python decorators are at minimal similar. There is a glaring difference too, Python decorators are  higher order functions. The decorator pattern is a software pattern. I should first answer the question what it is a higher-order function (for software patterns see that notebook). A higher-order function is a function that does at least one of the following:
 1. It takes one or more functions as arguments.
 2. It returns a function as its result.

Before we look at a decorator let's look at function composition, for function composition is an example of a higher-order function $f \circ g$  or $g(f(x))$.
The three standard operations on a sequence type (tuple, list, and array); map, filter, and fold (aka reduce) are higher-order functions. Let's see them in action.


In [1]:
tuple(map(lambda x : x**2, range(1,11)))

(1, 4, 9, 16, 25, 36, 49, 64, 81, 100)

In [2]:
tuple(filter(lambda x : x > 50, tuple(map(lambda x : x**2, range(1,11)))))

(64, 81, 100)

In [3]:
from functools import reduce
from operator import mul

# multiplying the numbers bigger than 50. 64 x 81, 100
reduce(mul,tuple(filter(lambda x : x > 50, tuple(map(lambda x : x**2, range(1,11))))))

518400

#### **Closure**
The decorator functions as a closure, a closure is a nested function that references one or more variables from its enclosing scope. This definition probably makes more sense with an example.

In the code below `display` is the closure. nested within `say`, `display` uses `greeting` which is defined outside of the local scope of `display`. However, greeting is part of `say()` which is also encompassing scope for `display`.

The variable with name greeting stays "alive" even if display does nothing.


In [13]:
def say():
    greeting = 'Hello'

    def display():
        print(greeting)
        

    return display()    

In [14]:
say()

Hello



You can access the closure directly, the interpreter knows of the existence, but trying to, does not lead to a result, as `display` need `say` to function.

In [24]:
display() 

## **decorators**
Formally closures make it is possible to give a sub-procedure one or more private variables that remain in existence between procedure calls. You actually see closures quite often in Python as you use a lot of decorators in Python. Decorators can be used in Python as higher-order functions and the decorated function has access to the decorator, the decorator contains the closure. Let's look at an example:

In [26]:
def decorator(func):
    
    def wrapper_decorator(*args, **kwargs):
        # Do something before
        func(*args, **kwargs) # This the fst time
        # Do something after
        return func(*args, **kwargs) #This is the third time 
    return wrapper_decorator

@decorator
def george(name):
    '''This is George his function'''
    print('George is a wooly rhino!')
    return f"hi {name}!"

george('George')

George is a wooly rhino!
George is a wooly rhino!


'hi George!'

This simple decorator does only one thing, it executes the print statement twice.

There is simple boilerplate code that allows you to write any decorator you want:

In [29]:
import functools

def decorator(func): # 1
    @functools.wraps(func) # 2
    def wrapper_decorator(*args, **kwargs): # 3 
        #  4 do something
        return func(*args, **kwargs) # 5
    return wrapper_decorator # 6

#### Code comment
 1. The name of the function to be used as the decorator, here aptly called decorator.
 2. The decorator at `@functools.wraps` ensures that you can inspect your decorated function and not the decorator.
 3. The convention is to call these functions wrapper_decorator_name, which in our case is decorator.
 4. Room to add to the functionality of the decorated function.
 5. If we want the function to do something else outside of the decorator, we need to return the original function with original arguments.
 6. Return for the decorator.

On point three inspecting `george` would result into useless info without the wraps decorator.


In [30]:
george

<function __main__.decorator.<locals>.wrapper_decorator(*args, **kwargs)>

In [31]:
help(george)

Help on function wrapper_decorator in module __main__:

wrapper_decorator(*args, **kwargs)



In [32]:
from functools import wraps

def decorator(func):
    @wraps(func)
    def wrapper_decorator(*args, **kwargs):
        # Do something before
        func(*args, **kwargs) # This the fst time
        func(*args, **kwargs) # This is the snd time
        # Do something after
        return func(*args, **kwargs) #This is the third time 
    return wrapper_decorator

@decorator
def george(name):
    '''This is George's his function'''
    print('George is a rhino!')
    return f"hi {name}!"

In [33]:
george

<function __main__.george(name)>

In [34]:
help(george)

Help on function george in module __main__:

george(name)
    This is George's his function



The primary use of decorators is to enrich functions with outside behaviour. 

Consider the simple Fibonacci function below which returns us the n-th Fibonacci number.

In [1]:
ok = 10 
waisting_time = 40
are_you_crazy = 1000

In [2]:
def fibonacci(n:int)->int:
    if n == 0: return 0
    if n == 1: return 1
    else: return fibonacci(n-1) + fibonacci(n-2)  
fibonacci(ok)

55

As we can see it works, but unfortunately this is not a very efficient function, which we see if we compute the 40th Fibonacci number.

In [3]:
%time fibonacci(waisting_time)

CPU times: total: 46.4 s
Wall time: 46.5 s


102334155

The inefficiency is in that we don't store intermediate results after all we have already computed the eight fibonacci number (34 in 34 + 55  = 89).
We could apply a bit of tabulation and store the intermediate results.

In [4]:
def fibonacci(n:int)->int:
    '''fibonacci with tabulation'''
    table = [None]*(n+1)
    table[0] = 0
    table[1] = 1
    for index in range(2,n+1):
        table[index] = table[index-1] + table[index-2]
    return table[n]
%time fibonacci(waisting_time)

CPU times: total: 0 ns
Wall time: 0 ns


102334155

This will surely stop wasting all that time by recalculating intermediate results, however it is a bit opaque and verbose.

There is a simple decorator in the functools library with which we enrich our original function to store intermediate results. With caching I can compute the 1000th Fibonacci number in a flash.

Below the surface this uses a similar technique as the example above. It caches (stores) intermediate results.


In [6]:
from functools import cache

@cache
def fibonacci(n:int)->int:
    '''this fibonacci method returns the n-th fibonacci number'''
    if n == 0: return 0
    if n == 1: return 1
    else: return fibonacci(n-1) + fibonacci(n-2)  
%time fibonacci(are_you_crazy)

CPU times: total: 0 ns
Wall time: 0 ns


43466557686937456435688527675040625802564660517371780402481729089536555417949051890403879840079255169295922593080322634775209689623239873322471161642996440906533187938298969649928516003704476137795166849228875

#### **Library decorators**
Python has many libraries that offer function decorators. Functools for instance, functools offers a function with which we can overload methods, something Python as a dynamically typed language does not support natively. See https://en.wikipedia.org/wiki/Function_overloading.

The idea behind overloaded functions is that you let the compiler decide which one to use at runtime. To be able to make the distinction which function to use the compiler needs to know two things: 
 1. It needs to know this function is overloaded, the decorator `@singledispatch` takes care of this.
 2. It needs to register the different implementations `@fun.register` takes care of that.


In [7]:
from functools import singledispatch

@singledispatch
def fun(arg, verbose=False):
    if verbose:
        print("Let me just say,", end=" ")
    print(arg)

In [12]:
@fun.register
def _(arg: int, verbose=False)->None:
    if verbose:
        print("Strength in numbers, eh?", end=" ")
    print(arg)


@fun.register
def _(arg: list, verbose=False)->None:
    if verbose:
        print("Enumerate this:")
    for i, elem in enumerate(arg):
        print(i+1, elem)

In [10]:
fun(1,True)

Strength in numbers, eh? 1


In [11]:
fun(['Ente', 'Rhino', 'Croc', 'George'], True)

Enumerate this:
1 Ente
2 Rhino
3 Croc
4 George



As you see the only difference between the two `fun` functions is the type of their argument. This is the only thing the compiler needs to choose between these functions. 

Other libraries also function decorators. One library that you will use a lot is Pytest, it knows decorators.

Run the code outside of this notebook, as notebooks usually are tested with doctest.

In [13]:
import pytest

@pytest.mark.parametrize("test_input,expected", [("3+5", 8), ("2+4", 6), ("6*9", 42)])
def test_eval(test_input, expected):
    assert eval(test_input) == expected
       

#### **Class decorators**
You are not bound to only use decorators on functions or methods, but you can also use a decorator on a class.

Consider the following code by now familiar code:

In [2]:
from functools import wraps

def croc(func):   
    @wraps(func)
    def wrapper_george(*args, **kwargs):
        # Do something before
        print('I am Peckish')
        func(*args, **kwargs) 
        # Do something after
        print('I need a snacky')
        return func(*args, **kwargs)
    return wrapper_george

@croc
def george(name)->str:
    print('This is George his function')
    return f'{name} did you know that George is a rhino?'

george('Lolo')

I am Peckish
This is George his function
I need a snacky
This is George his function


'Lolo did you know that George is a rhino?'

It was quite easy for Croc to wrap George's function, but what if George is a class with several methods?

Of course, we could opt to decorate all methods individually, but that is surely not very DRY (don't repeat yourself). We can do better we decorate the entire class. 

In [3]:
def crocs_class_decorator(cls): # 1
    for name, val in vars(cls).items(): # 2
        if callable(val): # 3 
            setattr(cls, name, croc(val)) # 4
    return cls

#### Code comment
 1. notice how a class is in Python essentially just an object. In Python we  can use an object as a first-class citizen: it can be used as an argument, it can be modified in a function, and can be used as a return value. If you are used to other languages this design simplicity should be appreciated. It is the reason why in my humble opinion Python is a much better programming language than it gets credits for.
 2. vars(cls) returns the `__dict__` attribute for a module, class, instance, or any other object with a `__dict__` attribute. `items` is a standard method of dictionaries. 
 3. callable returns True if the argument is callable, that is, if the argument implements the `__call__` method. Methods, function, classes, and modules are all callable.
 4. If the value is a callable (class or method) we wrap it with our decorator `croc(val)` and reinject it in the same class via the `setattr` function, now all callables in that class will be decorated.


In [4]:
@crocs_class_decorator
class MultipleGeorge:
    
    def george1(self, name:str)->str:
        print('George is a rhino!')
        return f"hi {name}!"
    
    def george2(self, name:str)->str:
        print('George is a woolly rhino!')
        return f"hi {name}!"
    
    def george3(self, name:str)->str:
        print('George is a grey rhino!')
        return f"hi {name}!"
    
    def george4(self, name:str)->str:
        print('George is a physicist & a rhino!')
        return f"hi {name}!"

In [5]:
mg = MultipleGeorge()
mg.george2('Croc')

I am Peckish
George is a woolly rhino!
I need a snacky
George is a woolly rhino!


'hi Croc!'

In [6]:
mg.george4('Croc')

I am Peckish
George is a physicist & a rhino!
I need a snacky
George is a physicist & a rhino!


'hi Croc!'

#### **Data classes**
With Python 3.7 introduced the `dataclasses` module with the @dataclass decorator I would advocate the use of data classes for four reasons:
 1. Data classes bring consistency in developing classes.
 2. Data classes solve the empty list as default value problem.
 3. Data classes allow for the easy use of Python's special methods and thus make better use of the Python data model in your code.
 4. Immutability of objects.
 
If you are interested in the second point I would advise you to read the Objects&Classes notebook, at the end I explain what the empty list as default value problem is, and how dataclasses solve it.

Here I want to foremostly focus on the first point, third, and fourth point. Any Python programmer knows at least a few ways with which we can program a class in Python. 

In [39]:
class ExampleOne:
    
    def __init__(self, att_one, att_two, att_three):
        self.att_one   = att_one
        self.att_two   = att_two
        self.att_three = att_three     
        
        def __str__(self):
            return f'This class {type(self).__name__} is written without the dataclasses module'
        
class ExampleTwo:
    
    def __init__(self):
        self.att_one   = 1
        self.att_two   = 'two'
        self.att_three = True     
        
        def __str__(self):
            return f'This class {type(self).__name__} is written without the dataclasses module'
        
class ExampleThree:
    att_one = 1
    att_two  = 'two'
    att_three = True  
    
    def __str__(self):
            return f'This class {type(self).__name__} is written without the dataclasses module'
    
class ExampleFour:
    
    def __init__(self, att_one=1, att_two='two', att_three=True):
        self.att_one   = att_one
        self.att_two   = att_two
        self.att_three = att_three    
        
    def __str__(self):
        return f'This class {type(self).__name__} is written without the dataclasses module'
        

you can even create a class using the `type` built-in function which can operate as class factory.

In [40]:
def __str__(self):
    return f'This class {type(self).__name__} is written without the dataclasses module'

ExampleFive = type('ExampleFive', (), {'att_one':1, 'att_two':'two','att_three':True, '__str__':__str__})
five = ExampleFive()
print(five)

This class ExampleFive is written without the dataclasses module


In [41]:
five.att_three

True


The first four are quite common ways to see a class created in Python. The last manner is a form of meta programming, where there is at least one more manner with which we can create a class in Python, and probably more I do not know about. These different ways of creating classes are confusing, and not only for beginning programmers. Furthermore, I suggest that having that many possibilities to create a class inevitably leads to faults.

Python data classes present a fifth (outside of the meta programming) manner with which to create a class. Data classes provide what can be best viewed as a framework for building classes consistently. A Python data class uses typed fields to hold the data attributes (and class attributes) that represent state and has methods to change that state. Data classes thus follow the theoretical class model more closely, which makes sure that programmers from different object-oriented languages understand a Python class intuitively. As important, dataclasses provide consistency, in class development.


In [19]:
from dataclasses import dataclass, field

@dataclass
class ExampleFive:
    #attributes
    att_one:int
    att_two:str
    att_three:bool
    
    #methods
    def __str__(self):
        return f'This class {type(self).__name__} is written with the dataclasses module'
    

In [20]:
five = ExampleFive(att_one=1, att_two='two', att_three=True)
print(five)

This class ExampleFive is written with the dataclasses module


#### **Typing**
As Eric V Smith writes in PEP 557: data classes can be thought of as “mutable namedtuples with defaults”. Data classes don't use the instance and class variables that you are used to. Instead, data classes use fields. A field is an attribute with type. The interpreter checks this when a class is created but does not enforce that the instance object uses the stated types (that would make Python a static language, it never will be, according to Guido van Rossum). Data classes, however, allow you to create a "static" version of Python, by using a type checker like MyPy, see https://mypy.readthedocs.io/en/stable/. But remember that you can still run the code even if MyPy signals the wrong types are used. In static typed languages like Haskell & Java the compiler would quite literally bite your head off.

Perhaps a bit odd Python throws a NameError if you define a field without a type.


In [17]:
from dataclasses import dataclass, field

@dataclass
class ExampleSix:
    #attributes
    att_one
    att_two:str
    att_three:bool
    
    #methods
    def __str__(self):
        return f'This class {type(self).__name__} is written with the dataclasses module'

NameError: name 'att_one' is not defined

In [18]:
five = ExampleFive(3.7, 0, 'George')
five

ExampleFive(att_one=3.7, att_two=0, att_three='George')

You can use default values in classes with fields, but you need to make sure that fields with default values come after fields with no values. If you want an empty list, set or tuple as default value, you need to use the field factory function from the `dataclasses` module.

You can also use class variables; you just need to annotate them as such.


In [42]:
from dataclasses import dataclass, field
from typing import ClassVar

@dataclass
class ExampleSix:
    #attributes
    att_one:int
    att_two:str
    att_three:bool
    
    #class variables
    cvar:ClassVar[int] = 0
    
    # default values
    att_four:list[str] = field(default_factory=list)
    att_five:set[int] = field(default_factory=set)
    
    #methods
    def __str__(self):
        return f'This class {type(self).__name__} is written with the dataclasses module'

In [43]:
six = ExampleSix(1, 'George', True)
six.__dict__

{'att_one': 1,
 'att_two': 'George',
 'att_three': True,
 'att_four': [],
 'att_five': set()}

In [44]:
six.__dataclass_fields__

{'att_one': Field(name='att_one',type=<class 'int'>,default=<dataclasses._MISSING_TYPE object at 0x0000020C6362AD10>,default_factory=<dataclasses._MISSING_TYPE object at 0x0000020C6362AD10>,init=True,repr=True,hash=None,compare=True,metadata=mappingproxy({}),kw_only=False,_field_type=_FIELD),
 'att_two': Field(name='att_two',type=<class 'str'>,default=<dataclasses._MISSING_TYPE object at 0x0000020C6362AD10>,default_factory=<dataclasses._MISSING_TYPE object at 0x0000020C6362AD10>,init=True,repr=True,hash=None,compare=True,metadata=mappingproxy({}),kw_only=False,_field_type=_FIELD),
 'att_three': Field(name='att_three',type=<class 'bool'>,default=<dataclasses._MISSING_TYPE object at 0x0000020C6362AD10>,default_factory=<dataclasses._MISSING_TYPE object at 0x0000020C6362AD10>,init=True,repr=True,hash=None,compare=True,metadata=mappingproxy({}),kw_only=False,_field_type=_FIELD),
 'cvar': Field(name='cvar',type=typing.ClassVar[int],default=0,default_factory=<dataclasses._MISSING_TYPE object 

#### **Immutability**
Immutability is one of the core principles of functional programming. It refers to the property that an entity can't be modified after being instantiated. Immutability allows you to write safer, cleaner code,  easier to test. After all, if an object cannot change its state everything that depends on that object can trust that state to remain the same.

In Python you cannot create truly immutable objects, after all Python is an objected-oriented language, not a functional one. Objects of course are all about changing state, and to create immutable objects seems very counter intuitive. Still not all objects need to be mutable after instantiation. Consider an email user class.


In [23]:
from dataclasses import dataclass

@dataclass
class EmailUser:
    user:str
    address:str
    

In [34]:
croc_mail = EmailUser(user='Croc', address='iampeckesh@feedme.com')
croc_mail

EmailUser(user='Croc', address='iampeckesh@feedme.com')

In [35]:
id(croc_mail)

2252261955280

The question now becomes do you think that if croc decides to change his email address to:

`ineedablueheron@snackies.com` 

The old croc_mail object should stay or that we should do:


In [36]:
del croc_mail

In [37]:
croc_mail = EmailUser(user='Croc', address='ineedablueheron@snackies.com')
croc_mail

EmailUser(user='Croc', address='ineedablueheron@snackies.com')

In [38]:
id(croc_mail)

2252289623952

To answer this question, you should see the bigger picture. Objects in large programs often depend on information in other objects.

If we have an object x that depends on the information of croc_mail.address to be `iampeckesh@feedme.com` and we change the address the x object might not perform its function and we have created a bug. Of course, you might say that the x object should depend on the croc_mail object, but that would be high coupling, which is an objected-oriented design faux pas.

An email user is uniquely identifiable by its email address, after all we have many a John Smith, yet only one john.smith@protonmail.com. To change the address is to fundamentally change the email user. You should delete the object and create a new one. 

With data classes I can create immutable objects, trying to change a field will result in a FrozenInstanceError.


In [45]:
from dataclasses import dataclass

@dataclass(frozen=True)
class EmailUser:
    user:str
    address:str


In [47]:
croc_mail = EmailUser(user='Croc', address='iampeckesh@feedme.com')
croc_mail

EmailUser(user='Croc', address='iampeckesh@feedme.com')

In [48]:
croc_mail.address='ineedablueheron@snackies.com'

FrozenInstanceError: cannot assign to field 'address'

#### **The Python data model**
Dataclasses allow for the easy use of Python's special methods. Special methods allow you to better leverage the Python Data Model in your programming. 

The use of special methods allows you to:
 1. Make better use the rich Python standard library and the Python Data Model.
 2. Standard operations can be used, called with standard function and can have have the same name throughout classes.
 3. Create clear and concise API's that other programmers will understand, because you build them on familiar methods. 

You have of course observed that all example classes have `__str__` method. This is a special method, Python has 80 of them. You don't call these 80 special methods yourself, they are linked with functions like `len` and `print` or operators (which are also functions of course) like `+` and `%`. Other programmers, serious Python developers, expect you to have used these special methods. 

#Dataclasses gives you access to all of them without having to inherit them. No base classes or metaclasses are used by Data Classes. Users of these classes are free to use inheritance and metaclasses without any interference from dataclasses. The decorated classes are truly “normal” Python classes. The Data Class decorator should not interfere with any usage of the class.

Dataclasses comes with many special methods predefined for you, eg. `__repr__` or `__data__`. So that you can call on a clear `help` functions for your classes or a clear `repr`. Others like `__hash__` you would have to implement yourself.

For more information I will refer you to PEP 557 https://peps.python.org/pep-0557/ & https://docs.python.org/3/reference/datamodel.html#.

Below is an example how we can leverage special methods and do operator overloading in a class we defined. 

In [108]:
from dataclasses import dataclass,field

@dataclass
class Vector:
    '''a vector in multidimensional space'''
    dim:int
    coordinates:list[int]= field(default_factory=list)
    
    def __post_init__(self)->None:
        '''creates a zero vector of the correct dimensions, directly after initialization'''
        self.coordinates = self.dim * [0]

    def __len__(self)->int:
        '''returns the dimensionality of the vector, which can now be called by using the built-in len function''' 
        return self.dim
    
    def __getitem__(self,i:int)->int:
        '''returns the i-th coordinate, start counting on 0'''
        if 0 > i or i > self.dim:
            raise ValueError(f'this vector has {self.dim} dimensions')
        return self.coordinates[i]
        
    
    def __setitem__(self,i:int, value:int)->None:
        '''set the i-th coordinate from 0'''
        if 0 > i or i > self.dim:
            raise ValueError(f'this vector has {self.dim} dimensions')
        if type(value)!=int:
            raise ValueError('the coordinates of the vectors need to be an int')
        self.coordinates[i] = value
        
    def fill_vector(self, values:list[int])->None:
        for i in range(self.dim):
            self[i]=values[i]
    
    def __add__(self, other:Vector)->Vector:
        if self.dim != len(other):
            raise ValueError('The vector dimensions must agree')
        result = Vector(self.dim)
        for i in range(self.dim):
            result[i] = self[i] + other[i]
        return result
    
    def __ne__(self, other:Vector)->bool:
        return not all([self[i]==other[i] for i in range(self.dim)])
    
    def __eq__(self, other:Vector)->bool:
        return all([self[i]==other[i] for i in range(self.dim)])
    
    def __str__(self)->str:
        return f'({str(self.coordinates)[1:-1]})'
        
    
   

#### Code comment
This class leverages several special methods and implements the sequence class, meaning that we can use the following built-in functions and operators:
 * `len`
 * `print`
 * indexing and slicing
 * `+`
 * `!=` 
 * `==`
 * we can iterate over it
 
just plays with it

In [109]:
v = Vector(3)
print(v)

(0, 0, 0)


In [105]:
for c in v:
    print(c)

0
0
0


---
#### **The end**